In [1]:
import os 


In [ ]:

from dotenv import load_dotenv 
load_dotenv  

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [3]:

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")



In [4]:
LANGCHAIN_PROJECT


'pr-excellent-treasury-98'

In [5]:

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT



In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")



/Users/preetyrai/Langgraph/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
'''from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")'''
from langchain_groq import ChatGroq
import os
llm=ChatGroq(model_name="Gemma2-9b-It")



# Simple AI Assistant

In [ ]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break
    


goodbye take care yourself


In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage


In [10]:
store={}


In [ ]:

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id] 



In [12]:
config = {"configurable": {"session_id": "firstchat"}}


In [13]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)


In [14]:
model_with_memory.invoke(("Hi! I'm sunnysavita"),config=config).content


"Hello Sunnysavita! It's nice to meet you. \n\nIs there anything I can help you with today? 😊  \n\n"

In [15]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content


'Your name is Sunnysavita!  🌻  \n\nI remember from our first introduction. 🙂  \n\n\n\n\n'

In [16]:
store 

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm sunnysavita", additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Sunnysavita! It's nice to meet you. \n\nIs there anything I can help you with today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 18, 'total_tokens': 50, 'completion_time': 0.058181818, 'prompt_time': 0.001940137, 'queue_time': 0.245079353, 'total_time': 0.060121955}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--a22e3ca4-b10d-41c9-bd43-bb8de09abff4-0', usage_metadata={'input_tokens': 18, 'output_tokens': 32, 'total_tokens': 50}), HumanMessage(content='tell me what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Sunnysavita!  🌻  \n\nI remember from our first introduction. 🙂  \n\n\n\n\n', additional_kwargs={}, response_metadata={'token_usage': {

# RAG WITH LCEL

In [17]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser


In [18]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

embeddings 

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x10eaa3d90>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [33]:
### Reading the txt files from source directory


# loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
# docs = loader.load()

# ### Creating Chunks using RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=50,
#     chunk_overlap=10,
#     length_function=len
# )
# new_docs = text_splitter.split_documents(documents=docs)
# doc_strings = [doc.page_content for doc in new_docs]

# ###  BGE Embddings
# '''

# from langchain.embeddings import HuggingFaceBgeEmbeddings

# model_name = "BAAI/bge-base-en-v1.5"
# model_kwargs = {'device': 'cpu'}
# encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
# embeddings = HuggingFaceBgeEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs,
# )

# '''  

# ### Creating Retriever using Vector DB

# db = Chroma.from_documents(new_docs, embeddings)
# retriever = db.as_retriever(search_kwargs={"k": 4})



In [21]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

# Step 1: Load documents
loader = DirectoryLoader('data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

# Step 2: Split documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)

if not new_docs:
    raise ValueError("No documents were found or split. Please check your file paths and content.")

# Step 3: Set up embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Step 4: Create vector store
db = Chroma.from_documents(new_docs, embeddings)

# Step 5: Set up retriever
retriever = db.as_retriever(search_kwargs={"k": 4})



In [22]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)



In [23]:

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )


In [24]:
question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))



Here are 3 important points about Llama 3 based on the provided text:

* **Llama 3 is a large language model.** This is implied by the descriptions of its various versions (8B parameters) and its use in services.
* **Llama 3 was released in April 2024.**  This is a specific detail provided in the text.
* **Llama 3 models are accessible outside a specific website.** This suggests Llama 3 is available for broader use and not just limited to a particular platform. 


Let me know if you have any other questions! 



# Let's start with Tools and Agents

In [25]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [26]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=100)



In [27]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)



In [28]:
tool.name


'wikipedia'

In [29]:
tool.description


'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [30]:
tool.args


{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [31]:
tool.return_direct


False

In [32]:
print(tool.run({"query": "langchain"}))



Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


In [34]:
tool.run("langchain")


'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of '

In [35]:
from langchain_core.pydantic_v1 import BaseModel, Field


/Users/preetyrai/Langgraph/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [38]:
# class WikiInputs(BaseModel):
#     query: str = Field(description="query to look up in Wikipedia, should be 3 or less words")



from pydantic import BaseModel

class WikiInputs(BaseModel):
    query: str

In [39]:
tool=WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)



In [40]:
tool.name



'wiki-tool'

In [41]:
tool.description



'look up things in wikipedia'

In [42]:
tool.args



{'query': {'title': 'Query', 'type': 'string'}}

In [43]:
tool.return_direct



True

In [44]:
print(tool.run("langchain"))



Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


# Youtube Search Tool

In [45]:
from langchain_community.tools import YouTubeSearchTool


In [46]:
tool=YouTubeSearchTool()


In [47]:
tool.name


'youtube_search'

In [48]:
tool.description


'search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional'

In [49]:
tool.run("sunny savita")


"['https://www.youtube.com/watch?v=3ZauAhbTyBE&pp=ygUMc3Vubnkgc2F2aXRh', 'https://www.youtube.com/watch?v=cydntJObPLI&pp=ygUMc3Vubnkgc2F2aXRh']"

In [50]:
from langchain_community.tools.tavily_search import TavilySearchResults


In [51]:
tool = TavilySearchResults()


In [52]:
tool.invoke({"query": "What happened in the latest burning man floods"})


[{'title': 'The latest on the Burning Man flooding - NPR',
  'url': 'https://www.npr.org/2023/09/03/1197497458/the-latest-on-the-burning-man-flooding',
  'content': "National\n\nThe latest on the Burning Man flooding\n\nBy\n\nClaudia Peschiutta\n\nThe latest on the Burning Man flooding\n\nAuthorities are investigating a death at the Burning Man festival in the Nevada desert after tens of thousands of people are stuck in camps because of rain. [...] Knee-deep mud, warnings to conserve food and water, orders to shelter in place - this is all at Burning Man 2023 after torrential rains turned the Black Rock Desert into miles and miles of mud. There are also reports that at least one person has died at the counterculture festival about a hundred miles north of Reno, Nev. Earlier this afternoon, I caught up with NPR's Claudia Peschiutta, who's at her first burn, and she told me it's muddy where she is, but that she and her camp family have been [...] Knee-deep mud, warnings to conserve food 

In [86]:
from langchain.agents import AgentType 
from langchain.agents import load_tools 
from langchain.agents import initialize_agent 



In [87]:
tool = load_tools(["wikipedia"], llm=llm) 


In [89]:
agent = initialize_agent(tool, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True) 
 


In [90]:
agent.run("what is current GDP of india")


/var/folders/kx/d1gxzt7s111d0z9bjypgv2980000gn/T/ipykernel_76662/3153669907.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("what is current GDP of india")




> Entering new AgentExecutor chain...
Thought: I need to look up the current GDP of India.
Action: wikipedia
Action Input: GDP of India
Observation: Page: Economy of India
Summary: The economy of India is a developing mixed economy with a notable public sector in strategic sectors. It is the world's fourth-largest economy by nominal GDP and the third-largest by purchasing power parity (PPP); on a per capita income basis, India ranked 136th by GDP (nominal) and 119th by GDP (PPP). From independence in 1947 until 1991, successive governments followed the Soviet model and promoted protectionist economic policies, with extensive Sovietization, state intervention, demand-side economics, natural resources, bureaucrat-driven enterprises and economic regulation. This is characterised as dirigism, in the form of the Licence Raj. The end of the Cold War and an acute balance of payments crisis in 1991 led to the adoption of a broad economic liberalisation in India and indicative planning. India

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `Thought:Thought: Thought: I need to look up the current GDP of India.



You have already asked about the current GDP of India.

Thought: I need to look up the current GDP of India.

Thought: Thought: I need to look up the current GDP of India.

Thought: Thought: I need to look up the current GDP of India.

Thought: I need to look up the current GDP of India.



I need to look up the current GDP of India.



Finally, I need to look up the current GDP of India.



Page:
Thought: I need to look up the current GDP of India.



Page: I need to look up the current GDP of India.



Please provide me with the current GDP of India.



Thought: I need to look up the current GDP of India.



Please provide me with the current GDP of India.



Please provide me with the current GDP

In [53]:
from langchain import hub


In [54]:
from langchain.agents import AgentExecutor,create_openai_functions_agent


In [55]:
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)



In [56]:
tavily_tool = TavilySearchResults()


In [57]:
tools = [tavily_tool]


In [58]:
agent = create_openai_functions_agent(llm, tools, prompt)



In [59]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)



In [60]:
print(agent_executor.invoke({"input": "who was the mahatma gandhi"}))





> Entering new AgentExecutor chain...


> Finished chain.
{'input': 'who was the mahatma gandhi', 'output': ''}


# Showcase that one more agents also

# Create our custom agents and custom tools

In [61]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from pydantic import BaseModel


/Users/preetyrai/Langgraph/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [62]:
@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"


In [63]:
print(search.name)
print(search.description)
print(search.args)


search
Look up things online.
{'query': {'title': 'Query', 'type': 'string'}}


In [64]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b



In [65]:
print(multiply.name)
print(multiply.description)
print(multiply.args)


multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [66]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")
    

In [67]:

@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"



In [68]:
print(search.name)
print(search.description)
print(search.args)
print(search.return_direct)


search-tool
Look up things online.
{'query': {'title': 'Query', 'type': 'string'}}
True


/Users/preetyrai/Langgraph/venv/lib/python3.10/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='should be a search query' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [69]:
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)


In [70]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")
    
    

In [71]:
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

    

# Here is my Custom Tool

In [72]:
from langchain.agents import tool
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


In [73]:
get_word_length.invoke("abc")


3

In [74]:
tools = [get_word_length]


In [75]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)



In [76]:
llm_with_tools = llm.bind_tools(tools)



In [77]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser



In [78]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)



In [79]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



In [80]:
llm.invoke("How many letters in the word educa")



AIMessage(content='There are **6** letters in the word "educa". \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 16, 'total_tokens': 33, 'completion_time': 0.030909091, 'prompt_time': 0.001913897, 'queue_time': 0.24846320300000002, 'total_time': 0.032822988}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--41b3bf72-5c5d-4cf3-af9f-db9aeb4c8d56-0', usage_metadata={'input_tokens': 16, 'output_tokens': 17, 'total_tokens': 33})

In [81]:
from langchain_core.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)



In [82]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []



In [83]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)



In [84]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)



In [85]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})





> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


55

> Finished chain.


> Entering new AgentExecutor chain...
Yes, "educa" is a real word. 





> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='5', additional_kwargs={}, response_metadata={})],
 'output': 'Yes, "educa" is a real word. \n\n\n\n'}